In [1]:
import tensorflow as tf

In [2]:

a = tf.keras.datasets.mnist.load_data()

print(type(a))

<class 'tuple'>


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [5]:
x_train[:1].shape

model(x_train[:1])

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[ 0.4712839 , -0.21485287,  0.12630862,  0.2416442 , -0.22935921,
        -0.09647187, -0.4055012 ,  0.21103227,  0.37569714, -0.94350904]],
      dtype=float32)>

In [6]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.4712839 , -0.21485287,  0.12630862,  0.2416442 , -0.22935921,
        -0.09647187, -0.4055012 ,  0.21103227,  0.37569714, -0.94350904]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [7]:
tf.nn.softmax(predictions).numpy()

array([[0.1560445 , 0.07857113, 0.11051662, 0.12402728, 0.07743958,
        0.08844541, 0.06493301, 0.12028808, 0.1418194 , 0.03791505]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [8]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [9]:
print(y_train[:1])

[5]


In [10]:
loss_fn([5],predictions).numpy()

2.42537

In [11]:
loss_fn(y_train[:1], predictions).numpy()

2.42537

In [12]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [13]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 1s 646us/step - loss: 0.3002 - accuracy: 0.9127
Epoch 2/5
1875/1875 [==============================] - 1s 629us/step - loss: 0.1447 - accuracy: 0.9569
Epoch 3/5
1875/1875 [==============================] - 1s 619us/step - loss: 0.1088 - accuracy: 0.9667
Epoch 4/5
1875/1875 [==============================] - 1s 618us/step - loss: 0.0880 - accuracy: 0.9725
Epoch 5/5
1875/1875 [==============================] - 1s 626us/step - loss: 0.0781 - accuracy: 0.9758


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [14]:
model(x_test)

<tf.Tensor: shape=(10000, 10), dtype=float32, numpy=
array([[ -3.5377545 ,  -9.128797  ,  -1.1373023 , ...,   9.562653  ,
         -2.3072207 ,  -0.83659226],
       [ -3.462874  ,   6.1743116 ,  11.86062   , ..., -12.093698  ,
         -2.6168025 , -13.652267  ],
       [ -6.1699834 ,   6.96991   ,  -2.258881  , ...,  -0.11411671,
         -2.7526343 ,  -6.381999  ],
       ...,
       [-10.535635  ,  -7.950803  ,  -6.5133553 , ...,   2.04795   ,
         -1.631945  ,   1.7945404 ],
       [ -4.501548  ,  -8.232307  ,  -9.6067505 , ...,  -6.0931444 ,
          2.57574   , -10.589997  ],
       [ -1.7841358 , -13.32701   ,  -2.7202668 , ..., -11.1556835 ,
         -9.02844   ,  -5.7942424 ]], dtype=float32)>

In [15]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0761 - accuracy: 0.9763


[0.07605002075433731, 0.9763000011444092]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [16]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [17]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.0413797e-06, 7.6166735e-09, 2.2512677e-05, 1.4120942e-03,
        1.0874242e-10, 2.8131979e-06, 1.4878032e-12, 9.9852318e-01,
        6.9877633e-06, 3.0410540e-05],
       [2.2060581e-07, 3.3805992e-03, 9.9660945e-01, 8.6141290e-06,
        2.4483610e-15, 7.6700964e-08, 5.1250925e-07, 3.9382032e-11,
        5.1411587e-07, 8.2874523e-12],
       [1.9630102e-06, 9.9886274e-01, 9.8060184e-05, 6.8156965e-06,
        9.6165364e-05, 3.0379280e-05, 5.0478766e-06, 8.3743647e-04,
        5.9849135e-05, 1.5879805e-06],
       [9.9998438e-01, 9.7663037e-11, 4.3243131e-07, 2.7453970e-07,
        7.5769485e-10, 1.0758499e-06, 1.0919385e-05, 9.2224741e-07,
        3.3491009e-08, 1.8518791e-06],
       [1.1019477e-05, 9.9434683e-09, 5.7513244e-06, 3.2917444e-07,
        9.9092591e-01, 3.5835771e-07, 4.4001827e-06, 1.4022206e-03,
        1.4298467e-06, 7.6486836e-03]], dtype=float32)>

In [20]:
type(x_test)

numpy.ndarray